<font size='4'><b>Notebook content:</b> An end-to-end solution to the Data Incubator's spark mini-project.</font>

</br></br>

In [0]:
# Importing libraries.
import datetime as dt
import numpy as np
import pyspark
import re
from collections import namedtuple
from lxml import etree

from pyspark import SparkContext, SparkConf
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import Word2Vec, HashingTF, Tokenizer, StopWordsRemover, CountVectorizer, IDF, RegexTokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.sql import functions as F
from pyspark.sql import SQLContext
from pyspark.sql.window import Window 

In [0]:
print(sc)

<SparkContext master=local[8] appName=Databricks Shell>

### Bad XML

In [0]:
# Setting the path.
import os
def localpath(path):
    return 'file://' + str(os.path.abspath(os.path.curdir)) + '/' + path

In [0]:
# Parsing function to remove bad XML.
def row_start(line_current):
    line_strip = line_current.strip().strip('\n')
    return (line_strip.find("<row ")==0) 

In [0]:
#lines = sc.textFile(localpath('/spark-stats-data/allPosts/'))
start_rows = lines.filter(lambda x: row_start(x))
rows=lines.filter(lambda x: is_row(x))

In [0]:
rows.count()

Out[6]: 9763

In [0]:
# Calculating bad XML count.
abs(rows.count()-start_rows.count())

Out[11]: 237

In [0]:
# Feed to the grader through this function.
def bad_xml():
    return 237

<br/><br/>

### Favorites and score

In [0]:
#def is_row(line_current):
 #   line_strip = line_current.strip().strip('\n')
  #  return (line_strip.find("<row ")==0 and line_strip[-3:]==" />")

In [0]:
# Do it at the very beginning as it will restart the python kernel.
%pip install lxml

Python interpreter will be restarted.
Collecting lxml
 Downloading lxml-4.6.3-cp37-cp37m-manylinux1_x86_64.whl (5.5 MB)
Installing collected packages: lxml
Successfully installed lxml-4.6.3
Python interpreter will be restarted.

In [0]:
from collections import namedtuple

In [0]:
def is_row(line_current):
    line_strip = line_current.strip().strip('\n')
    return (line_strip.find("<row ")==0 and line_strip[-3:]==" />")

In [0]:
# Reading posts data into an RDD.
rdd_posts = sc.textFile('/FileStore/tables/part_00000_xml.gz')
rdd_posts.count()

Out[11]: 20506

In [0]:
# Function to parse scores.
def fetch_scores(line): 
    
    #line_encoded = line.strip().encode('utf-8')
    named_tuple = namedtuple('named_tuple', ['FavoriteCount','score','count'])
    root = etree.fromstring(line)
    
    try: 
        FavoriteCount = int(root.attrib['FavoriteCount'])
    except: 
        FavoriteCount = int(0)
    
    try: 
        score = int(root.attrib['Score'])
    except: 
        score = int(0)
    
    single = named_tuple(FavoriteCount,score, 1)
    
    return  (single.FavoriteCount, (single.score, 1))

In [0]:
# Function to parse posts.
def parse_posts(line):
    line = line.encode('utf-8') 
    try: 
        root = etree.fromstring(line)
        return etree.tostring(root)
    except: 
        return None

In [0]:
parsed_posts = rdd_posts.filter(lambda x: is_row(x)) \
               .map(parse_posts) \
               .filter(lambda x: x is not None)

In [0]:
# Combining scores and posts.
combined = parsed_posts.map(fetch_scores)\
                .reduceByKey(lambda x,y: tuple(np.add(x,y)))\
                .sortByKey()\
                .take(50)

In [0]:
combined

[(0, (43683, 7652)), (1, (2879, 657)), (2, (2435, 437)), (3, (1828, 243)), (4, (1394, 163)), (5, (1075, 116)), (6, (974, 79)), (7, (892, 67)), (8, (696, 50)), (9, (715, 44)), (10, (524, 28)), (11, (381, 21)), (12, (429, 21)), (13, (317, 15)), (14, (226, 11)), (15, (264, 12)), (16, (280, 12)), (17, (284, 10)), (18, (214, 10)), (19, (169, 6)), (20, (88, 4)), (21, (160, 5)), (22, (209, 7)), (23, (274, 8)), (24, (142, 5)), (25, (101, 2)), (26, (109, 3)), (27, (256, 5)), (28, (99, 3)), (29, (201, 4)), (30, (83, 2)), (31, (172, 5)), (32, (36, 1)), (33, (73, 2)), (34, (76, 2)), (35, (82, 2)), (36, (89, 2)), (37, (109, 2)), (38, (154, 3)), (39, (120, 3)), (40, (47, 1)), (41, (51, 1)), (42, (52, 1)), (45, (64, 1)), (47, (78, 1)), (48, (60, 1)), (49, (47, 1)), (52, (76, 1)), (54, (78, 1)), (58, (91, 2))]

<br/><br/>

### Answer percentage

In [0]:
rdd_posts = sc.textFile(localpath('/spark-stats-data/allPosts/')) 
rdd_posts.count()

212990

In [0]:
rdd_users = sc.textFile(localpath('/spark-stats-data/allUsers/'))
rdd_users.count()

100425

In [0]:
# Function to parse users.
def parse_users(line):
    if '  <row'in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "Id" and "Reputation" in root.attrib:
                return(root.attrib["Id"], int(root.attrib["Reputation"]))
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")

In [0]:
parsed_users = rdd_users.map(parse_users).filter(lambda x: x!= 'Empty')
parsed_users.count()

50320

In [0]:
# Function to parse posts.
def parse_posts(line):
    if '  <row'in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "OwnerUserId" and "PostTypeId" in root.attrib:
                type = root.attrib["PostTypeId"]
                try:
                    owner = root.attrib["OwnerUserId"]
                except:
                    pass
                    return ("Empty")
                if type == '2':
                    type_count = [1.,0.,1.]
                else:
                    if type == '1':
                        type_count = [0.,1.,1.]
                    else:
                        return("Empty")
                return(owner, type_count)
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")

In [0]:
parsed_posts = rdd_posts.map(parse_posts).filter(lambda x: x!= 'Empty')\
        .reduceByKey(lambda x, y: np.add(x,y))

parsed_posts.count()

Out[69]: 1521

In [0]:
parsed_posts = rdd_posts.map(parse_posts).filter(lambda x: x!= 'Empty')\
        .reduceByKey(lambda x, y: np.add(x,y))

parsed_posts.count()

26887

In [0]:
# Combining posts and users.
combined = parsed_posts.join(parsed_users).map(lambda x: (x[1][1], x[1][0], int(x[0])))\
        .map(lambda x: (x[0],(x[2], x[1][0]/x[1][2]))).sortByKey(False)\
        .map(lambda x: x[1]).collect()
print(combined[:99])

[(919, 0.996694214876033), (805, 0.9959749552772809), (686, 0.9803049555273189), (7290, 0.9918887601390498), (930, 0.9817351598173516), (4505, 1.0), (4253, 0.9909747292418772), (183, 0.847870182555781), (11032, 0.9875647668393782), (28746, 0.968421052631579), (887, 0.9794871794871794), (159, 0.9728813559322034), (2116, 0.9833333333333333), (4856, 0.9543147208121827), (22047, 1.0), (5739, 0.9872773536895675), (3277, 0.956081081081081), (88, 0.9660493827160493), (2970, 1.0), (601, 0.9772151898734177), (17230, 0.9970059880239521), (449, 1.0), (2392, 0.9724137931034482), (1390, 0.9411764705882353), (5836, 0.846441947565543), (7555, 1.0), (603, 0.8158844765342961), (7972, 0.9823008849557522), (6633, 0.9912280701754386), (2958, 0.9930313588850174), (9394, 0.9700854700854701), (7828, 0.9850427350427351), (2817, 0.8206896551724138), (7224, 0.9757575757575757), (4598, 0.9857142857142858), (7071, 0.9107142857142857), (1739, 0.9948717948717949), (1036, 0.9545454545454546), (3382, 1.0), (8013, 0.9040697674418605), (3019, 0.8571428571428571), (4376, 0.963302752293578), (251, 0.9924242424242424), (28666, 0.9), (1764, 0.9325842696629213), (23853, 1.0), (32036, 0.9959839357429718), (10849, 0.9518072289156626), (26338, 0.9691358024691358), (1352, 0.9902912621359223), (401, 0.9119496855345912), (5, 0.8547008547008547), (8, 0.8991596638655462), (7250, 0.9877300613496932), (1909, 0.9518072289156626), (21054, 0.9345794392523364), (4257, 0.9757575757575757), (196, 0.7357512953367875), (442, 0.8712121212121212), (279, 1.0), (2669, 0.946843853820598), (8402, 0.6521739130434783), (36041, 0.9889807162534435), (2126, 1.0), (44269, 0.9033613445378151), (6029, 1.0), (11981, 0.9649122807017544), (1934, 0.9680851063829787), (795, 0.676056338028169), (25433, 0.9867256637168141), (253, 0.3695652173913043), (364, 0.6736111111111112), (25, 0.9166666666666666), (22311, 0.9401709401709402), (334, 1.0), (13047, 0.9733333333333334), (8507, 0.9428571428571428), (264, 0.90625), (14188, 0.8983050847457628), (307, 1.0), (8076, 0.9333333333333333), (5862, 1.0), (8413, 0.9836065573770492), (1307, 0.8333333333333334), (2860, 0.8903225806451613), (223, 0.8588235294117647), (11887, 0.976878612716763), (52554, 0.9652777777777778), (2074, 1.0), (35989, 0.9487179487179487), (1005, 0.0034482758620689655), (22228, 0.8513513513513513), (4862, 0.8974358974358975), (3601, 0.9852941176470589), (17908, 1.0), (13138, 0.8639455782312925), (1108, 0.9722222222222222), (1679, 0.941747572815534), (11852, 1.0)]

In [0]:
# Feed to the grader in this format.
total = parsed_posts.map(lambda x: ("t",x[1])).reduceByKey(lambda x, y: np.add(x,y)).collect()
sum_of_total = total[0]
fraction = (-1, sum_of_total[1][0]/sum_of_total[1][2])
print(fraction)

(-1, 0.5157639985252271)

<br/><br/>

### First Question

In [0]:
rdd_users = sc.textFile('/FileStore/tables/user_part_00000_xml.gz')
rdd_users.count()

Out[2]: 99515

In [0]:
rdd_posts = sc.textFile('/FileStore/tables/part_00000_xml.gz') 
rdd_posts.count()

Out[3]: 20506

In [0]:
rdd_users = sc.textFile(localpath('/spark-stats-data/allUsers/'))
rdd_users.count()

100425

In [0]:
rdd_posts = sc.textFile(localpath('/spark-stats-data/allPosts/')) 
rdd_posts.count()

212990

In [0]:
def is_row(line_current):
    line_strip = line_current.strip().strip('\n')
    return (line_strip.find("<row ")==0 and line_strip[-3:]==" />")

In [0]:
# Function to parse users.
def parse_users(line): 
    
    line= line.encode('utf-8')
    named_tuple = namedtuple('named_tuple', ['userID','reputation', 'user_creation_date'])
    
    try: 
        root = etree.fromstring(line)
    except: 
        root= None
    
    try: 
        userID = root.attrib['Id']
    except: 
        userID = None
    
    try: 
        reputation = int(root.attrib['Reputation'])
    except: 
        reputation = 0
    
    try: 
        date_object = root.attrib['CreationDate'].split('T')
        date_concat = date_object[0] + ' ' + date_object[1]
        user_creation_date = dt.datetime.strptime(date_concat, '%Y-%m-%d %H:%M:%S.%f') 
    except: 
        user_creation_date = None
    
    single = named_tuple(userID, reputation, user_creation_date)
    return single


In [0]:
parsed_users= rdd_users.filter(lambda x: is_row(x))\
                    .map(parse_users) \
                    .filter(lambda x: x[0] != None and x[2] != None) \
                    .map(lambda x: (x.userID, (x.reputation, x.user_creation_date)))

In [0]:
# Function to parse posts.
def parse_posts(line):
    line = line.encode('utf-8') 
    try: 
        root = etree.fromstring(line)
        return etree.tostring(root)
    except: 
        return None

In [0]:
# Function to extract dates.
def fetch_dates(line):
    
    #row = row.encode('utf-8')
    named_tuple_posts = namedtuple('named_tuple_posts', ['userID','question', 'question_creation_date'])
    
    try: 
        root = etree.fromstring(line)
    except: 
        root = None
        
    try: 
        userID = root.attrib['OwnerUserId']
    except: 
        userID = None   
    
    try: 
        if root.attrib['PostTypeId'] == '1':
            question = 'Yes'
        else: 
            question = None
    except: 
        question = None
    
    try: 
        date_object = root.attrib['CreationDate'].split('T')
        date_concat = date_object[0] + ' ' + date_object[1]
        question_creation_date = dt.datetime.strptime(date_concat, '%Y-%m-%d %H:%M:%S.%f')
    except: 
        question_creation_date = None
    
    single = named_tuple_posts(userID, question, question_creation_date)
    return single    
    

In [0]:
parsed_posts = rdd_posts.filter(lambda x: is_row(x)) \
               .map(parse_posts) \
               .filter(lambda x: x is not None)

In [0]:
# Combining post creation date with posts.
dates .parsed_posts_date = parsed_posts.map(fetch_dates) \
                .filter(lambda x: x[0] != None and x[1] != None and x[2] != None) \
                .map(lambda x: (x.userID, x.question_creation_date))

In [0]:
# Comparing user creation date against post creation date.
combined = parsed_posts_date.reduceByKey(lambda x, y: min(x, y)) \
                        .join(parsed_users) \
                        .map(lambda x: (int(x[0]), (x[1][0] - x[1][1][1]).days, x[1][1][0])) \
                        .takeOrdered(100, key=lambda x: -x[2])

In [0]:
combined

Out[13]: [(919, 3, 100976),
 (686, 190, 47334),
 (930, 4, 32283),
 (183, 0, 23610),
 (887, 22, 20315),
 (159, 0, 20133),
 (3277, -1, 16131),
 (88, 0, 14768),
 (601, 19, 14100),
 (2392, 42, 12491),
 (1390, 117, 12098),
 (603, 41, 11830),
 (2958, 63, 11083),
 (2817, 0, 10552),
 (1036, 5, 9530),
 (3019, 0, 8794),
 (251, 2, 8221),
 (1764, 1, 7971),
 (1352, 0, 7552),
 (401, 34, 7116),
 (5, 0, 6962),
 (8, 0, 6948),
 (1909, 87, 6814),
 (196, 0, 6682),
 (442, 28, 6588),
 (2669, 20, 6352),
 (1934, 0, 5967),
 (795, 0, 5849),
 (253, 12, 5762),
 (364, 7, 5739),
 (25, 0, 5661),
 (264, 70, 5085),
 (1307, 3, 4238),
 (2860, 0, 4204),
 (223, 1, 4192),
 (1005, 42, 4080),
 (1108, 65, 3805),
 (1679, 2, 3747),
 (2645, 0, 3646),
 (1381, 0, 3572),
 (1569, 0, 3369),
 (26, 213, 3220),
 (1124, 10, 3195),
 (339, 4, 3179),
 (2310, 0, 3124),
 (3567, 9, 3106),
 (485, 14, 3022),
 (215, 0, 2751),
 (1106, 0, 2640),
 (29, 105, 2570),
 (2775, 1, 2530),
 (3911, 0, 2501),
 (74, 0, 2496),
 (582, 0, 2450),
 (656, 165, 2340),
 (2798, 10, 2336),
 (30, 7, 2330),
 (438, 1, 2328),
 (696, 103, 2317),
 (4426, 0, 2267),
 (3262, 0, 2235),
 (521, 9, 2213),
 (561, 20, 2147),
 (439, 92, 2143),
 (1347, 0, 2100),
 (2485, 15, 2092),
 (319, 1, 2089),
 (71, 0, 2071),
 (776, 0, 2025),
 (61, 0, 2024),
 (666, 22, 1995),
 (199, 0, 1955),
 (511, 1, 1916),
 (1540, 25, 1844),
 (39, 0, 1741),
 (1329, 4, 1739),
 (4479, 0, 1697),
 (1670, 78, 1679),
 (36, 210, 1625),
 (3874, 43, 1600),
 (179, 77, 1561),
 (3488, 31, 1545),
 (144, 0, 1486),
 (273, 12, 1469),
 (3748, 1, 1455),
 (226, 59, 1385),
 (230, 180, 1374),
 (2719, -1, 1363),
 (333, 0, 1353),
 (557, 70, 1348),
 (1499, 0, 1342),
 (2425, 44, 1342),
 (266, 0, 1332),
 (89, 1, 1311),
 (3919, 0, 1282),
 (1913, -1, 1250),
 (851, 0, 1225),
 (190, 0, 1196),
 (2921, 0, 1196),
 (881, 38, 1188)]

</br></br>

### Identify Veterans

In [0]:
# Function to parse users.
def parse_users(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "Id" and "CreationDate" in root.attrib:
                return(root.attrib["Id"], root.attrib["CreationDate"])
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")

In [0]:
parsed_users = rdd_users.map(parse_users).filter(lambda x: x!= 'Empty')
parsed_users.count()

50320

In [0]:
# Function to parse posts.
def parse_posts(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")            
        if root != '':
            if "CreationDate" in root.attrib:
                try:
                    return (root.attrib["OwnerUserId"], root.attrib["CreationDate"])
                except:
                    pass
                    return ("Empty")
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")

In [0]:
parsed_posts = rdd_posts.map(parse_posts).filter(lambda x: x!= 'Empty')

parsed_posts.count()

107152

In [0]:
# Function to parse questions.
def parse_questions(line):
    if '<row'in line:
        try:
            root = ET.fromstring(line.encode('utf8').strip())
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "OwnerUserId" in root.attrib:
                p_type = root.attrib["PostTypeId"]
                owner = root.attrib["OwnerUserId"]
                if p_type == '1':
                    score = float(root.attrib["Score"])
                    views = float(root.attrib["ViewCount"])
                    answers = float(root.attrib["AnswerCount"])
                    create_date = root.attrib["CreationDate"] 
                    favs = 0.
                    try:
                        favs = float(root.attrib["FavoriteCount"])
                    except:
                        pass

                    return(owner, [create_date, score, views, answers, favs])
                else:
                    return("Empty")
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")

In [0]:
parsed_questions = rdd_posts.map(parse_questions).filter(lambda x: x!= "Empty")\
            .reduceByKey(min).map(lambda x: (x[0],x[1][1:]))
parsed_questions.count()

23133

In [0]:
parsed_questions.take(10)

[('23994', [-1.0, 33.0, 1.0, 0.0]), ('61518', [4.0, 173.0, 1.0, 1.0]), ('35549', [1.0, 390.0, 1.0, 0.0]), ('36313', [3.0, 73.0, 0.0, 0.0]), ('69951', [0.0, 15.0, 0.0, 0.0]), ('11549', [2.0, 304.0, 1.0, 1.0]), ('60913', [1.0, 12.0, 0.0, 0.0]), ('17525', [3.0, 65.0, 2.0, 0.0]), ('11384', [1.0, 264.0, 0.0, 0.0]), ('43270', [0.0, 69.0, 1.0, 0.0])]

In [0]:
from datetime import datetime
import time

In [0]:
# Function to find whether user has posted between 100 and 150 days.
def date_conversion (date1, date2):  
    obj1= datetime.strptime(date1, "%Y-%m-%dT%H:%M:%S.%f")
    obj2= datetime.strptime(date2, "%Y-%m-%dT%H:%M:%S.%f")
    diff = obj2- obj1
    diff_sec = diff.total_seconds()
    if diff_sec > 8640000 and diff_sec < 12960000:
        return 1
    else:
        return 0

In [0]:
# More than 1 post is veteran otherwise brief user.
def transform(x):
    if x[1]>=1:
        veteran =1
    else:
        veteran= 0
    return (x[0],veteran)

In [0]:
# Combining users and posts.
combined = parsed_users.join(parsed_posts).map(lambda x: (x[0], date_conversion(x[1][0], x[1][1])))\
        .reduceByKey(lambda x,y: x+y).map(transform)

In [0]:
combined.take(10)

[('23991', 0), ('55619', 0), ('55344', 0), ('35549', 0), ('22465', 0), ('69953', 0), ('13732', 1), ('11549', 0), ('28999', 0), ('4028', 0)]

In [0]:
# Combining classification and questions.
veterans = combined.join(parsed_questions).map(lambda x: (x[1][0], x[1][1]+[1]))\
            .reduceByKey(lambda x,y: np.add(x,y))\
            .map(lambda x: (x[0], [x[1][0]/x[1][4], x[1][1]/x[1][4], x[1][2]/x[1][4], x[1][3]/x[1][4]]))

In [0]:
veterans.take(5)

Out[47]: [(0,
 [7.491609081934847,
 2919.062191510365,
 2.4777887462981245,
 3.5735439289239883]),
 (1,
 [10.171270718232044,
 3450.453038674033,
 2.9060773480662982,
 5.574585635359116])]

</br></br>

### Identify veterans for full data/stack

In [0]:
rdd_posts = sc.textFile(localpath('/spark-stack-data/allPosts/')) 
rdd_posts.count()

47946654

In [0]:
rdd_users = sc.textFile(localpath('/spark-stack-data/allUsers/')) 
rdd_users.count()

7978096

In [0]:
# Function to parse users.
def parse_users(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "Id" and "CreationDate" in root.attrib:
                return(root.attrib["Id"], root.attrib["CreationDate"])
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")

In [0]:
parsed_users = rdd_users.map(parse_users).filter(lambda x: x!= 'Empty')
parsed_users.count()

4016017

In [0]:
parsed_users.take(10)

[('2629093', '2013-07-29T06:01:42.410'), ('2629094', '2013-07-29T06:02:00.537'), ('2629095', '2013-07-29T06:02:44.543'), ('2629096', '2013-07-29T06:04:16.690'), ('2629097', '2013-07-29T06:04:34.343'), ('2629098', '2013-07-29T06:04:58.327'), ('2629099', '2013-07-29T06:05:24.410'), ('2629100', '2013-07-29T06:05:31.910'), ('2629101', '2013-07-29T06:05:33.473'), ('2629102', '2013-07-29T06:05:49.413')]

In [0]:
# Function to parse posts.
def parse_posts(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")            
        if root != '':
            if "CreationDate" in root.attrib:
                try:
                    return (root.attrib["OwnerUserId"], root.attrib["CreationDate"])
                except:
                    pass
                    return ("Empty")
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")

In [0]:
parsed_posts = rdd_posts.map(parse_posts).filter(lambda x: x!= 'Empty')

parsed_posts.count()

23875001

In [0]:
parsed_posts.take(20)

[('2433934', '2014-04-22T06:15:07.070'), ('1692695', '2014-04-22T06:15:07.190'), ('2655465', '2014-04-22T06:15:33.113'), ('3189313', '2014-04-22T06:15:37.650'), ('3330929', '2014-04-22T06:15:46.763'), ('3557296', '2014-04-22T06:15:59.837'), ('692977', '2014-04-22T06:16:00.200'), ('3557098', '2014-04-22T06:16:03.460'), ('2616474', '2014-04-22T06:16:04.017'), ('2020444', '2014-04-22T06:16:05.267'), ('133802', '2014-04-22T06:16:11.333'), ('418864', '2014-04-22T06:16:11.633'), ('2846794', '2014-04-22T06:16:12.743'), ('1915323', '2014-04-22T06:16:12.957'), ('2541124', '2014-04-22T06:16:15.587'), ('1060208', '2014-04-22T06:16:19.550'), ('2156712', '2014-04-22T06:16:20.430'), ('2504224', '2014-04-22T06:16:21.340'), ('1446916', '2014-04-22T06:16:22.707'), ('3319147', '2014-04-22T06:16:26.467')]

In [0]:
# Function to parse questions.
def parse_questions(line):
    if '<row'in line:
        try:
            root = ET.fromstring(line.encode('utf8').strip())
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "OwnerUserId" in root.attrib:
                p_type = root.attrib["PostTypeId"]
                owner = root.attrib["OwnerUserId"]
                if p_type == '1':
                    score = float(root.attrib["Score"])
                    views = float(root.attrib["ViewCount"])
                    answers = float(root.attrib["AnswerCount"])
                    create_date = root.attrib["CreationDate"] 
                    favs = 0.
                    try:
                        favs = float(root.attrib["FavoriteCount"])
                    except:
                        pass

                    return(owner, [create_date, score, views, answers, favs])
                else:
                    return("Empty")
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")

In [0]:
parsed_questions = rdd_posts.map(parse_questions).filter(lambda x: x!= "Empty")\
            .reduceByKey(min).map(lambda x: (x[0],x[1][1:]))
parsed_questions.count()

1664063

In [0]:
parsed_questions.take(10)

[('400303', [0.0, 861.0, 1.0, 0.0]), ('3233915', [0.0, 63.0, 1.0, 0.0]), ('1394056', [1.0, 12115.0, 3.0, 0.0]), ('1988847', [1.0, 1832.0, 1.0, 0.0]), ('3382688', [1.0, 195.0, 1.0, 0.0]), ('4022777', [-2.0, 28.0, 2.0, 0.0]), ('4227558', [0.0, 16.0, 1.0, 0.0]), ('656672', [3.0, 450.0, 1.0, 0.0]), ('382512', [5.0, 13683.0, 3.0, 0.0]), ('2811225', [0.0, 679.0, 1.0, 0.0])]

In [0]:
# Function to find whether user has posted between 100 and 150 days.
from datetime import datetime
import time

def date_conversions(date1, date2):  
    obj1 = datetime.strptime(date1, "%Y-%m-%dT%H:%M:%S.%f")
    obj2= datetime.strptime(date2, "%Y-%m-%dT%H:%M:%S.%f")
    diff = obj2- obj1
    diff_sec = diff.total_seconds()
    if diff_sec > 8640000 and diff_sec < 12960000:
        return 1
    else:
        return 0

In [0]:
# More than 1 post is veteran otherwise brief user.
def transform(x):
    if x[1]>=1:
        veteran =1
    else:
        veteran = 0
    return (x[0],veteran)

In [0]:
# Combining users and posts.
combined = parsed_users.join(parsed_posts).map(lambda x: (x[0], date_conversion(x[1][0], x[1][1])))\
        .reduceByKey(lambda x,y: x+y).map(transform)
combined.count()

2134402

In [0]:
combined.take(40)

[('4244030', 0), ('3572161', 0), ('2366235', 0), ('3081058', 0), ('4517024', 0), ('226394', 0), ('3415433', 0), ('2067486', 0), ('2089439', 0), ('4533468', 0), ('4245037', 0), ('3172408', 1), ('49114', 0), ('2277308', 0), ('945718', 0), ('3808011', 0), ('2279515', 0), ('2171253', 0), ('1025276', 0), ('2528105', 1), ('3023252', 0), ('3271310', 0), ('2062661', 0), ('3433536', 0), ('2052956', 0), ('1096793', 0), ('472777', 0), ('904209', 0), ('3358938', 0), ('1834308', 0), ('1518873', 0), ('1554668', 1), ('2430620', 1), ('4569777', 0), ('3551380', 0), ('4546220', 0), ('794996', 1), ('2742088', 0), ('2005020', 0), ('3927882', 0)]

In [0]:
# Combining classification and questions.
veterans = combined.join(parsed_questions).map(lambda x: (x[1][0], x[1][1]+[1]))\
            .reduceByKey(lambda x,y: np.add(x,y))\
            .map(lambda x: (x[0], [x[1][0]/x[1][4], x[1][1]/x[1][4], x[1][2]/x[1][4], x[1][3]/x[1][4]]))

In [0]:
veterans.take(5)

[(0, [1.1307456144103445, 1096.1519220732553, 1.5038565525030159, 0.3861764445851408]), (1, [2.2598437331442924, 1844.0344896669696, 1.8426197044183144, 0.8673157237744455])]

</br></br>

###  Word2Vec for ggplot2.

In [0]:
# Setting vector size and random seed as per the problem statement.
word2vec = Word2Vec().setVectorSize(100).setSeed(42)

In [0]:
def is_row(line_current):
    line_strip = line_current.strip().strip('\n')
    return (line_strip.find("<row ")==0 and line_strip[-3:]==" />")

In [0]:
# Function to extract tags.
def fetch_tags(line):
    root = etree.XML(line)
    tags = []
    if 'Tags' in root.attrib: 
        Tag_string = root.get('Tags')
        tags = Tag_string.replace('&lt;', ' ').replace('&gt;', ' ').replace('<', ' ').replace('>', ' ').split()
    else:
        return None
    return tags

In [0]:
tags = sc.textFile(localpath('spark-stack-data/allPosts'))\
        .filter(is_row)\
        .map(fetch_tags)\
        .filter(lambda x: x is not None)

tags.count()

8966083

In [0]:
model = word2vec.fit(tags)

In [0]:
# Finding synonyms.
synonyms = model.findSynonyms('ggplot2', 25)

for word, cosine_distance in synonyms:
    print("{} {}".format(word, cosine_distance))

lattice 0.914175089362
r-grid 0.853013366773
plotmath 0.844089403526
boxplot 0.839753503564
plotrix 0.831518218593
ecdf 0.830968580308
gmisc 0.824988746884
levelplot 0.824531900738
density-plot 0.824492297613
melt 0.816001808733
gridextra 0.813623201172
line-plot 0.809634432145
loess 0.807918811886
rgl 0.804110386348
tapply 0.803792142344
ggvis 0.801683385182
mgcv 0.801153310268
r-factor 0.798146535195
quantile 0.797029189706
performanceanalytics 0.794787815957
weibull 0.792841571028
ggdendro 0.791935147398
categorical-data 0.790621594458
ggmap 0.790548134704
standard-error 0.788530333391

In [0]:
result='''
lattice 0.914175089362
r-grid 0.853013366773
plotmath 0.844089403526
boxplot 0.839753503564
plotrix 0.831518218593
ecdf 0.830968580308
gmisc 0.824988746884
levelplot 0.824531900738
density-plot 0.824492297613
melt 0.816001808733
gridextra 0.813623201172
line-plot 0.809634432145
loess 0.807918811886
rgl 0.804110386348
tapply 0.803792142344
ggvis 0.801683385182
mgcv 0.801153310268
r-factor 0.798146535195
quantile 0.797029189706
performanceanalytics 0.794787815957
weibull 0.792841571028
ggdendro 0.791935147398
categorical-data 0.790621594458
ggmap 0.790548134704
standard-error 0.788530333391
'''

</br></br>

### Classifications

In [0]:
def is_row(line_current):
    line_strip = line_current.strip().strip('\n')
    return (line_strip.find("<row ")==0 and line_strip[-3:]==" />")

In [0]:
train = sc.textFile('/FileStore/tables/part_00001').filter(lambda x: is_row(x))
test= sc.textFile('/FileStore/tables/part_00002').filter(lambda x: is_row(x))

In [0]:
train = sc.textFile(localpath('/spark-stats-data/train/')).filter(lambda x: is_row(x))
test= sc.textFile(localpath('/spark-stats-data/test/')).filter(lambda x: is_row(x))

In [0]:
# Function to extract tags.
def fetch_tags(row):

    row = row.strip().encode('utf-8')

    try: 
        root = etree.fromstring(row)
        post_id = int(root.attrib['Id'])
        tags = root.attrib['Tags']
        return (post_id, tags)  
    except Exception: 
        return None 

In [0]:
# Extracting the top 10 tags from training set.
alpha_numeric = re.compile('[a-zA-Z0-9-\']+') 
tags_10 = train.map(fetch_tags) \
          .filter(lambda x: x is not None) \
          .map(lambda line: (alpha_numeric.findall(line[1].lower()),)) \
          .flatMap(lambda x: [(tag, 1) for tags in x for tag in tags]) \
          .reduceByKey(lambda x,y: x+y) \
          .takeOrdered(10, key = lambda x: -x[1])

In [0]:
tags_10_words = [i[0] for i in tags_10]
tags_10_words

Out[38]: ['r',
 'regression',
 'machine-learning',
 'time-series',
 'probability',
 'distributions',
 'hypothesis-testing',
 'logistic',
 'correlation',
 'bayesian']

In [0]:
# Function check whether a given tag is in top 10 tags or not.
def within_top(line): 
    
    for tag in tags_10_words:
        if tag in line:
            return 1
    return 0

In [0]:
# Function to extract body and tag.
def fetch_body_and_tags(line):
    
    line = line.strip().encode('utf-8')
    
    try: 
        root = etree.fromstring(line)
        body = root.attrib['Body']
        tags = root.attrib['Tags']
        post_type = int(root.attrib['PostTypeId'])
        return (body, tags, post_type)  
    except Exception: 
        return None

In [0]:
training = train.map(fetch_body_and_tags) \
                                .filter(lambda x: x is not None and x[2] == 1) \
                                .map(lambda x: (x[0], alpha_numeric.findall(x[1].lower()))) \
                                .map(lambda x: (x[0], within_top(x[1]))) \
                                .toDF(['body', 'label']) \
                                .cache()
training.show(3)

+--------------------+-----+
 body|label|
+--------------------+-----+
<p>I am construct...| 1|
<p><strong>Edited...| 1|
<p>I have a data ...| 1|
+--------------------+-----+
only showing top 3 rows

In [0]:
# Function to extract body and ID.
def fetch_id_body(line):

    line = line.strip().encode('utf-8')
    
    try: 
        root = etree.fromstring(line)
        Id = int(root.attrib['Id'])
        body = root.attrib['Body']
        post_type = int(root.attrib['PostTypeId'])
        return (Id, body,post_type)   
    except Exception: 
        return None

In [0]:
testing = test.map(fetch_id_body) \
                        .filter(lambda x: x is not None and x[2] == 1) \
                        .toDF(['ID', 'body']) \
                        .select('ID', 'body')
testing.show(3)

+-----+--------------------+
 ID| body|
+-----+--------------------+
73939|<p>I've got some ...|
74047|<p>I know that th...|
74050|<p>I am trying to...|
+-----+--------------------+
only showing top 3 rows

In [0]:
tokenizer = Tokenizer(inputCol="body", outputCol="words")
stopwords_remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="words_minus_stopwords")
hashingTF = HashingTF(inputCol=stopwords_remover.getOutputCol(), outputCol="features", numFeatures = 2000)

In [0]:
train_tokens = tokenizer.transform(training)
train_tokens_nostopwords = stopwords_remover.transform(train_tokens)
train_hashes = hashingTF.transform(train_tokens_nostopwords)

In [0]:
test_tokens = tokenizer.transform(testing)
test_tokens_nostopwords = stopwords_remover.transform(test_tokens)
test_hashes = hashingTF.transform(test_tokens_nostopwords)

In [0]:
# Building a logistic regression model.
logreg = LogisticRegression(maxIter=10, regParam=0.08)
model = logreg.fit(train_hashes)

In [0]:
prediction = model.transform(test_hashes)
selected = prediction.select("ID", "body", "prediction")

In [0]:
selected_sorted = selected.sort("ID").collect()
selected_list = [int(x[2]) for x in selected_sorted]
selected_list[:10]

Out[50]: [1, 0, 1, 0, 1, 1, 1, 0, 0, 1]

In [0]:
classification = selected_list
classification[:20]

Out[51]: [1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0]

### Model tuning

In [0]:
tokenizer = Tokenizer(inputCol="body", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

tokens = tokenizer.transform(training)
hashes = hashingTF.transform(tokens)

test_tokens = tokenizer.transform(testing)
test_hashes = hashingTF.transform(test_tokens)

In [0]:
pipeline = Pipeline(stages=[tokenizer, hashingTF, logreg])

In [0]:
paramGrid = (ParamGridBuilder()  
    .addGrid(hashingTF.numFeatures, [2000])
    .addGrid(logreg.regParam, [0.5, 0.1, 0.05])
    .build())

In [0]:
# Cross validating through a pipeline.
cross_validator = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=6)

In [0]:
cvModel = cross_validator.fit(training)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [0]:
better_prediction = cvModel.transform(testing)
better_prediction.show(3)

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 ID| body| words| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
73939|<p>I've got some ...|[<p>i've, got, so...|(2000,[10,17,52,6...|[-0.5806087746260...|[0.35879252666476...| 1.0|
74047|<p>I know that th...|[<p>i, know, that...|(2000,[17,55,99,1...|[0.02844672286465...|[0.50711120118112...| 0.0|
74050|<p>I am trying to...|[<p>i, am, trying...|(2000,[15,17,43,4...|[-0.1461162703857...|[0.46353578513799...| 1.0|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 3 rows

In [0]:
better_selected = better_prediction.select("ID", "body", "prediction")
better_selected = better_selected.sort("ID")
better_selected.show(3)

+-----+--------------------+----------+
 ID| body|prediction|
+-----+--------------------+----------+
73939|<p>I've got some ...| 1.0|
74047|<p>I know that th...| 0.0|
74050|<p>I am trying to...| 1.0|
+-----+--------------------+----------+
only showing top 3 rows

<br/><br/>

<font size='4'><b>Final thoughts:</b> Whole approach boils down to two broader tasks: parsing the given XML files followed by applying suitable aggregations. In Word2Vec and classification problems, NLP/ML is also involved. Because data are huge, it is advisable to code for sample files and later scale it up to save time. Further, K-means problem is not solved because it is ungraded. </font> 